# Introduction to F# #

F# is an [open-source, cross-platform functional programming language](http://aka.ms/fsharphome) for .NET.

F# has features and idioms to support functional programming while also offering clean interop with C# and existing .NET codebases and systems. It can use anything in the [NuGet](https://www.nuget.org/) ecosystem, as this notebook will also demonstrate.

## F# basics

Let's start with some simple arithmetic:

In [1]:
(12/4 + 5 + 7) * 4 - 18

42

Arithmetic is nice, but there's so much more you can do. Here's how you can generate some data using the `[start .. end]` range syntax:

In [7]:
let numbers = [0 .. 10]
numbers

index,value
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


You can use slices with the `.[start .. end]` syntax to slice a subset of the data you just generated:

In [9]:
// Take the numbers from 2nd index to the 5th
numbers.[2..5]

index,value
0,2
1,3
2,4
3,5


And you can use indexer syntax (`.[index]`) to access a single value:

In [15]:
numbers.[3]

3

## Functions in F# #

Since F# is a functional language, functions are one of the first things to learn. You do that with the `let` keyword. F#, like Python, uses indentation to define code blocks:

In [11]:
let sampleFunction x =
    2*x*x - 5*x + 3

F# uses type inference to figure out types for you. But if needed, you can specify types explicitly:

In [12]:
let sampleFunction' (x: int) =
    2*x*x - 5*x + 3

When calling F# functions, parentheses are optional:

In [23]:
sampleFunction 5

28

In [14]:
sampleFunction' 12

231

You can define and compose F# functions easily:

In [21]:
let negate x = -x
let square x = x * x
let print x = printfn "The number is %d" x

let squareNegateThenPrint x =
    print (negate (square x))
    
squareNegateThenPrint 5

The number is -25


The pipeline operator `|>` is used extensively in F# code to chain functions and arguments together. It helps readability when building functional "pipelines":

In [22]:
// Redefine the function with pipelines
let squareNegateThenPrint x =
    x
    |> square
    |> negate
    |> print

squareNegateThenPrint 5

The number is -25


## Strings, tuples, lists, and arrays

Strings in F# use `"` quotations. You can concatenate them with the `+` operator:

In [25]:
let s1 = "Hello"
let s2 = "World"

s1 + ", " + s2 + "!"

Hello, World!

You can use triple-quoted strings (`"""`) if you want to have a string that contains quotes:

In [27]:
"""A triple-quoted string can contain quotes "like this" anywhere within it"""

A triple-quoted string can contain quotes "like this" anywhere within it

### Tuples

Tuples are simple combinations of data items into a single value. The following defines a tuple of an integer, string, and double:

In [28]:
(1, "fred", Math.PI)

Item1,Item2,Item3
1,fred,3.141592653589793


You can also create `struct` tuples when you have performance-sensitive environments:

In [30]:
struct (1, Math.PI)

Item1,Item2
1,3.141592653589793


### Lists

Lists are linear sequences of values of the same type. In fact, you've already seen them above when we generated some numbers!

In [31]:
[0 .. 10]

index,value
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


You can use list comprehensions to generate more advanced data programmatically:

In [38]:
let thisYear = DateTime.Now.Year

let fridays =
    [
        for month in 1 .. 10 do
            for day in 1 .. DateTime.DaysInMonth(thisYear, january) do
                let date = DateTime(thisYear, january, day)
                if date.DayOfWeek = DayOfWeek.Friday then
                    date.ToShortDateString()
    ]

// Get the first five fridays of this year
fridays
|> List.take 5

index,value
0,1/4/2019
1,1/11/2019
2,1/18/2019
3,1/25/2019
4,1/4/2019


Since you can slice lists, the first five fridays could also be done like this:

In [42]:
fridays.[..4]

index,value
0,1/4/2019
1,1/11/2019
2,1/18/2019
3,1/25/2019
4,1/4/2019


### Arrays

Arrays are very similar to lists. A key difference is that array internals are mutable. They also have better performance characteristics than lists.

In [43]:
let firstTwoHundred = [| 1 .. 200 |]
firstTwoHundred.[197..]

index,value
0,198
1,199
2,200


Processing lists and arrays is typically done by built-in and custom functions:

In [46]:
// Filter the previous list of numbers and sum their squares.
firstTwoHundred
|> Array.filter (fun x -> x % 3 = 0)
|> Array.sumBy (fun x -> x * x)

882189

## Types

Although F# is succinct, it actually uses static typing! Types are central to F# programming, especially when you want to model more complicated data to manipulate later in a program.

### Records

Record types are used to combine different kinds of data into an aggregate. They cannot be `null` and come with default comparison and equality.

In [47]:
type ContactCard =
    { Name: string
      Phone: string
      ZipCode: string }

// Create a new record
{ Name = "Alf"; Phone = "(555) 555-5555"; ZipCode = "90210" }

Name,Phone,ZipCode
Alf,(555) 555-5555,90210


You can access record labels with `.`-notation:

In [48]:
let alf = { Name = "Alf"; Phone = "(555) 555-5555"; ZipCode = "90210" }
alf.Phone

(555) 555-5555

Records are comparable and equatable:

In [51]:
// Create another record
let ralph = { Name = "Ralph"; Phone = "(123) 456-7890"; ZipCode = "90210" }

// Check if they're equal
alf = ralph

False

You'll find yourself writing functions that operate on records all the time:

In [58]:
let showContactCard contact =
    contact.Name + " - Phone: " + contact.Phone + ", Zip: " + contact.ZipCode
    
showContactCard alf

Alf - Phone: (555) 555-5555, Zip: 90210

### Discriminated Unions

Discriminated Unions (often called DUs) provide support for values that can be one of a number of names cases. These cases can be completely different from one another.

In the following example, we combine records with a discriminated union:

In [62]:
type Shape =
    | Rectangle of width : float * length : float
    | Circle of radius : float
    | Prism of width : float * float * height : float
    
let rect = Rectangle(length = 1.3, width = 10.0)
let circ = Circle (1.0)
let prism = Prism(5., 2.0, height = 3.0)

## Pattern matching

The best way to work with DUs is pattern matching. Using the previously-defined type definitions, we can model withdrawing money from an account:

In [69]:
let height shape =
    match shape with
    | Rectangle(width = h) -> h
    | Circle(radius = r) -> 2.0 * r
    | Prism(height = h) -> h
    
let rectHeight = height rect
let circHeight = height circ
let prismHeight = height prism

printfn "rect is %0.1f, circ is %0.1f, and prism is %0.1f" rectHeight circHeight prismHeight

rect is 10.0, circ is 2.0, and prism is 3.0


You can pattern match on more than just discriminated unions. Here we write a recursive function with `rec` to process lists:

In [76]:
// See if x is a multiple of n
let isPrimeMultiple n x =
    x = n || x % n <> 0
    
// Process lists recursively.
// '[]' means the empty list.
// 'head' is an item in the list.
// 'tail' is the rest of the list after 'head'.
let rec removeMultiples ns xs =
    match ns with
    | [] -> xs
    | head :: tail ->
        xs
        |> List.filter (isPrimeMultiple head)
        |> removeMultiples tail
        
let getPrimesUpTo n =
    let max = int (sqrt (float n))
    removeMultiples [2 .. max] [1 .. n]
    
// Primes up to 25
getPrimesUpTo 25

index,value
0,1
1,2
2,3
3,5
4,7
5,11
6,13
7,17
8,19
9,23


## Options

A built-in DU type is the F# option type. It is used prominently in F# code. Options can either be `Some` or `None`, and they're best used when you want to account for when there may not be a value.

In [78]:
let keepIfPositive a =
    if a > 0 then
        Some a
    else 
        None
        
keepIfPositive 12

Value
12


Options are often used when searching for values. Here's how you can incorporate them into list processing:

In [80]:
let rec tryFindMatch predicate lst =
    match lst with
    | [] -> None
    | head :: tail ->
        if predicate head then
            Some head
        else
            tryFindMatch predicate tail
          
let greaterThan100 x = x > 100

tryFindMatch greaterThan100 [25; 50; 100; 150; 200]

Value
150


## Parallel Programming

For more CPU-intensive tasks, you can take advantage of built-in parallelism:

In [82]:
let bigArray = [| 0 .. 100_000 |]

let rec compute x =
    if x <= 2 then
        1
    else
        compute (x - 1) + compute (x - 2)

// We'll use the F# formatter for these results, since they are enormous
let results =
    bigArray
    |> Array.Parallel.map (fun x -> compute (x % 24))

printfn "%A" results

[|1; 1; 1; 2; 3; 5; 8; 13; 21; 34; 55; 89; 144; 233; 377; 610; 987; 1597; 2584;
  4181; 6765; 10946; 17711; 28657; 1; 1; 1; 2; 3; 5; 8; 13; 21; 34; 55; 89; 144;
  233; 377; 610; 987; 1597; 2584; 4181; 6765; 10946; 17711; 28657; 1; 1; 1; 2; 3;
  5; 8; 13; 21; 34; 55; 89; 144; 233; 377; 610; 987; 1597; 2584; 4181; 6765;
  10946; 17711; 28657; 1; 1; 1; 2; 3; 5; 8; 13; 21; 34; 55; 89; 144; 233; 377;
  610; 987; 1597; 2584; 4181; 6765; 10946; 17711; 28657; 1; 1; 1; 2; ...|]


In [ ]:
let rec fib x = if x <= 2 then 1 else fib(x-1) + fib(x-2)
    
Array.Parallel.init 100 (fun i -> fib i)